In [1]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from mtcnn import MTCNN

import os

In [2]:
# loading face mask recognition model
maskNet = load_model("face_mask_detector.model")

In [3]:
def detect_and_predict_mask(frame, masknet):
    
    faces, locs, preds = [], [], []

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    results = detector.detect_faces(img)
    for result in results:
        if result['confidence']>0.5:
            print(result['confidence'])
            box = result['box']

            face = img[box[1]:box[1]+box[3], box[0]:box[0]+box[2]]
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective lists
            faces.append(face)
            locs.append((box[0], box[1], box[0]+box[2], box[1]+box[3]))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = masknet.predict(faces, batch_size=32)

    return (locs, preds)

In [4]:
frame = cv2.imread(r"C:\Users\tapen\Pictures\face_mask_detection_annoted_dataset\images\maksssksksss5.png")
# frame = cv2.imread('pic2.jpg')

(locs, preds) = detect_and_predict_mask(frame, maskNet)

# loop over the detected face locations and their corresponding
# locations
for (box, pred) in zip(locs, preds):
    # unpack the bounding box and predictions
    (startX, startY, endX, endY) = box
    (incorrect, mask, withoutMask) = pred

    # determine the class label and color we'll use to draw
    # the bounding box and text
    print(pred)
    if max(pred) == mask :
        label = "With Mask"
        color = (0, 255, 0)
    elif max(pred) == withoutMask:
        label = "Without Mask"
        color = (0, 0, 255)
    else:
        label = "Wearing Mask Incorrectly"
        color = (0, 255, 255)

    # include the probability in the label
    label = "{}: {:.2f}%".format(label, max(pred) * 100)

    # display the label and bounding box rectangle on the output frame
    cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX,0.4,(255,255,255),1)
    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

# show the output frame
cv2.imshow("Frame", frame)

k = cv2.waitKey(0)
if k == ord('q'):
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

0.9999997615814209
0.9999270439147949
0.9981012940406799
0.9969972372055054
0.7064287066459656
[1.7169102e-03 2.3051084e-04 9.9805260e-01]
[2.6668780e-04 2.0278508e-03 9.9770552e-01]
[0.00964502 0.9746878  0.01566717]
[0.14551875 0.8458261  0.00865513]
[3.7744903e-04 1.0097603e-03 9.9861276e-01]
